In [2]:
import sys
import os
import json
import pandas
import numpy
import optparse
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict
from keras.models import Sequential, load_model

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
df_black = pandas.read_csv('data/badqueries.txt',engine='python',sep='!@#$%^&*',header=0)
df_white = pandas.read_csv('data/goodqueries.txt',engine='python',sep='!@#$%^&*',header=0).sample(n=50000)

In [4]:
df_black['label'] = 1
df_white['label'] = 0

In [5]:
new_dataset = df_black.append(df_white)


In [6]:
new_dataset = new_dataset.sample(n=10000)

In [25]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(new_dataset)
print(tokenizer)

In [26]:
X_waf = new_dataset['url'].values.astype('str')
Y_waf = new_dataset['label'].values.astype('str')

In [27]:
max_log_length = 1024
X_sequences = tokenizer.texts_to_sequences(X_waf)
X_processed = sequence.pad_sequences(X_sequences, maxlen=max_log_length)

In [28]:
model = load_model('securitai-lstm-model.h5')
model.load_weights('securitai-lstm-weights.h5')
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [29]:
# Evaluate model
score, acc = model.evaluate(X_processed, Y_waf, verbose=1, batch_size=128)
print("Model Accuracy: {:0.2f}%".format(acc * 100))

10000/10000 [==============================] - 47s 5ms/step
Model Accuracy: 47.03%


In [18]:
prediction = model.predict(X_processed)
print(prediction[0])

[0.6021086]
